In [1]:
from mpi4py import MPI
from petsc4py import PETSc

from tqdm.notebook import tqdm_notebook

import numpy as np

import ufl

from dolfinx import fem, io, mesh, plot, la
from dolfinx.fem.petsc import assemble_matrix, create_vector, apply_lifting, NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from basix.ufl import element, mixed_element
from dolfinx.io import XDMFFile

import pyvista

In [2]:
# create mesh for calculations to take place in
msh = mesh.create_unit_square(MPI.COMM_WORLD, 16,16)

# define elements to be used for the velocity and the distribution
s_p1 = element("P", msh.basix_cell(), 1)
p_p1 = element("P", msh.basix_cell(), 1)
v_p2 = element("P", msh.basix_cell(), 2, shape=(msh.geometry.dim,))

# define time step variables 
T = 10        # final time
num_steps = 1000   # number of time steps
dt = T / num_steps # time step size

In [3]:
def initial_condition(x, a=10):
    return np.exp(-a * ((x[0]-.5)**2 + (x[1]-.5)**2))

# Function to mark x = 0, x = 1 and y = 0
def noslip_boundary(x):
    return np.logical_or(np.logical_or(np.isclose(x[0], 0.0), np.isclose(x[0], 1.0)),
                         np.isclose(x[1], 0.0))

# Function to mark the lid (y = 1)
def lid(x):
    return np.isclose(x[1], 1.0)

def flow(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1])))

In [4]:
TH = mixed_element([v_p2, p_p1])
W = fem.functionspace(msh, TH)

S = fem.FunctionSpace(msh, s_p1)

s_ = fem.Function(S)
s_n = fem.Function(S)

# s_.interpolate(initial_condition)
# s_n.interpolate(initial_condition)

s = ufl.TrialFunction(S)
z = ufl.TestFunction(S)

In [5]:
V, submapV = W.sub(0).collapse()
Q, submapQ = W.sub(1).collapse()

# No slip boundary condition
noslip = fem.Function(V)
facets = mesh.locate_entities_boundary(msh, 1, noslip_boundary)
dofs = fem.locate_dofs_topological((W.sub(0), V), 1, facets)
noslip_bc = fem.dirichletbc(noslip, dofs, W.sub(0))

# Driving velocity condition u = (1, 0) on top boundary (y = 1)
lid_velocity = fem.Function(V)
lid_velocity.interpolate(flow)
facets = mesh.locate_entities_boundary(msh, 1, lid)
dofs = fem.locate_dofs_topological((W.sub(0), V), 1, facets)
lid_flow_bc = fem.dirichletbc(lid_velocity, dofs, W.sub(0))

# distribution boundary conditions
inflow_dofs = fem.locate_dofs_geometrical(S, lid)
inflow_dist_bc = fem.dirichletbc(PETSc.ScalarType(1), inflow_dofs, S)
outflow_dist_bc = fem.dirichletbc(PETSc.ScalarType(-1), inflow_dofs, S)

# Collect Dirichlet boundary conditions
flow_bc = [noslip_bc, lid_flow_bc]
dist_bc = [outflow_dist_bc, inflow_dist_bc]


In [6]:
# Define variational problem
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)
f = fem.Function(V)

F1 = (ufl.inner(ufl.grad(u), ufl.grad(v)) + ufl.inner(p, ufl.div(v)) + ufl.inner(ufl.div(u), q)) * ufl.dx - ufl.inner(f, v) * ufl.dx # taylor hood
a1 = fem.form(ufl.lhs(F1))
L1 = fem.form(ufl.rhs(F1))


# Assemble LHS matrix and RHS vector
A1 = fem.petsc.assemble_matrix(a1, bcs=flow_bc)
A1.assemble()
b1 = fem.petsc.assemble_vector(L1)

fem.petsc.apply_lifting(b1, [a1], bcs=[flow_bc])
b1.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
fem.petsc.set_bc(b1, flow_bc)

In [1]:
# Create and configure solver
ksp = PETSc.KSP().create(msh.comm)
ksp.setOperators(A1)
ksp.setType("preonly")

# Configure MUMPS to handle pressure nullspace
pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("superlu_dist")

NameError: name 'PETSc' is not defined

In [8]:
flow_solution = fem.Function(W)

ksp.solve(b1, flow_solution.vector)

u_n = flow_solution.sub(0)

In [9]:
eps = fem.Constant(msh, PETSc.ScalarType(0.01)) # diffusion coefficent
rho = fem.Constant(msh, PETSc.ScalarType(0))
k = fem.Constant(msh, dt)
f = fem.Constant(msh, PETSc.ScalarType(0))

In [10]:
F = 0

F += ((s_ - s_n) / k) * z * ufl.dx # advective terms
F +=  ufl.dot( u_n, ufl.grad(s_)) * z *ufl.dx # advective terms
F += eps * ufl.dot( ufl.grad( s_ ), ufl.grad( z ) ) * ufl.dx # diffusive term
F -=  f * z  * ufl.dx


prob = NonlinearProblem(F, s_, bcs=dist_bc)

solver = NewtonSolver(MPI.COMM_WORLD, prob)


# a = fem.form(ufl.lhs(F))
# L = fem.form(ufl.rhs(F))

In [11]:
# A = assemble_matrix(a, bcs=bcs)
# A.assemble()
# b = create_vector(L)

# solver = PETSc.KSP().create(msh.comm)
# solver.setOperators(A)
# solver.setType(PETSc.KSP.Type.PREONLY)
# solver.getPC().setType(PETSc.PC.Type.LU)

In [12]:
outputfile = io.XDMFFile( msh.comm, "Reportfigures/DiffusionFlow.xdmf", "w")

outputfile.write_mesh( msh )

In [13]:
t = 0
outputfile.write_function( s_, t )

for i in tqdm_notebook( range( num_steps )):
    t += dt
    
    # with b.localForm() as loc_b:
    #     loc_b.set(0)
        
    # fem.petsc.assemble_vector(b, L)
    # apply_lifting(b, [a], [bcs])
    # b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    # fem.set_bc(b, bcs)
        
    # solver.solve(b, s_.vector)
    solver.solve(s_)
    
    s_.x.scatter_forward() # correct for inacuracies 

    s_n.x.array[:] = s_.x.array

    outputfile.write_function( s_, t )
outputfile.close()

  0%|          | 0/1000 [00:00<?, ?it/s]